# 提升方法

### 杜小勤，武汉纺织大学数学与计算机学院，2020年03月18日
https://github.com/duxiaoqin/Lecture-notes-for-Machine-Learning

AdaBoost算法的实现

In [1]:
import numpy as np
from collections import Counter
import copy

In [2]:
class AdaBoost:
    def __init__(self, base):
        self.base = base #弱分类器
        
    def fit(self, X, Y, max_step):
        N = len(X)
        self.W = np.ones(N) / N #每个样本的权值初始化
        self.alpha = [] #存放弱分类器权重
        self.weaker = [] #存放弱分类器
        for step in range(max_step): #训练弱分类器
            weaker = copy.deepcopy(self.base) #生成一个新弱分类器
            weaker.fit(X, Y, sample_weight = self.W) #使用弱分类器带样本权值进行训练
            results = weaker.predict(X)
            #输出准确度（Accuracy）
            scores = (results == Y)
            print('Weaker {:} accuracy = {:3.2f}%'.format(step, Counter(scores)[True]/len(Y) * 100))
            error = np.dot(self.W, [0 if score else 1 for score in scores]) #计算当前弱分类器的分类误差率：带权值误差
            alpha = 0.5 * np.log((1 - error)/error) #计算当前弱分类器的权重
            self.alpha.append(alpha)
            self.W = self.W * np.exp(alpha * np.array([-1 if score else 1 for score in scores])) #更新每个样本的权值：分类正确-yG(x)=-1，否则1
            self.W = self.W / sum(self.W) #归一化
            self.weaker.append(weaker)
        #计算最终分类器的结果
        f = np.dot(np.c_[[weaker.predict(X) for weaker in self.weaker]].T, np.array(self.alpha).reshape(-1, 1))
        results = [1 if y >= 0 else -1 for y in f]
        scores = (results == Y)
        print('\nAdaBoost accuracy = {:3.2f}%, training completed!'.format(Counter(scores)[True]/len(Y) * 100))
        
    def predict(self, X):
        f = np.dot(np.c_[[weaker.predict(X) for weaker in self.weaker]].T, np.array(self.alpha).reshape(-1, 1))
        return [1 if y >= 0 else -1 for y in f]

例8.1的程序验证

构建例8.1的数据集

In [3]:
X = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Y = [1, 1, 1, -1, -1, -1, 1, 1, 1, -1]

构造一个简易的弱分类器

In [4]:
import operator

In [5]:
class Simple: #最简决策树桩，仅用于验证例8.1
    def fit(self, X, Y, sample_weight):
        minloss = float('inf')
        for compare in [operator.le, operator.ge]:
            for threshold in X:
                results = compare(np.array(X), threshold)
                results = [1 if result else -1 for result in results]
                results = results == Y
                results = [0 if result else 1 for result in results]
                loss = np.dot(results, sample_weight)
                if minloss > loss:
                    minloss = loss
                    self.threshold = threshold
                    self.compare = compare
    def predict(self, X):
        return [1 if self.compare(x, self.threshold) else -1 for x in X]

In [6]:
ada = AdaBoost(Simple())
ada.fit(np.array(X), np.array(Y), 3)
print(ada.alpha)

Weaker 0 accuracy = 70.00%
Weaker 1 accuracy = 70.00%
Weaker 2 accuracy = 60.00%

AdaBoost accuracy = 100.00%, training completed!
[0.42364893019360172, 0.64964149206513044, 0.75203869838813697]


使用sklearn的决策树作为决策树桩

In [7]:
from sklearn.tree import DecisionTreeClassifier
ada = AdaBoost(DecisionTreeClassifier(max_depth = 1))
ada.fit(np.array(X).reshape(-1, 1), np.array(Y), 3)
print(ada.alpha)

Weaker 0 accuracy = 70.00%
Weaker 1 accuracy = 70.00%
Weaker 2 accuracy = 60.00%

AdaBoost accuracy = 100.00%, training completed!
[0.42364893019360172, 0.64964149206513044, 0.75203869838813697]


使用鸢尾花数据集进行测试，载入预存的鸢尾花数据集

In [8]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
iris_npz = np.load('iris.npz')
data = iris_npz['data']
X = iris_npz['X']
Y = iris_npz['Y']
#转换成适合AdaBoost处理的标签
Y[:50] = 1
Y[50:] = -1

In [10]:
XTRAIN, XTEST, YTRAIN, YTEST = train_test_split(X, Y, test_size = 0.25)

In [11]:
import copy

ada = AdaBoost(DecisionTreeClassifier(max_depth = 1)) #使用决策树桩作为弱分类器
ada.fit(XTRAIN, YTRAIN, 10)

Weaker 0 accuracy = 90.67%
Weaker 1 accuracy = 84.00%
Weaker 2 accuracy = 77.33%
Weaker 3 accuracy = 64.00%
Weaker 4 accuracy = 74.67%
Weaker 5 accuracy = 70.67%
Weaker 6 accuracy = 86.67%
Weaker 7 accuracy = 80.00%
Weaker 8 accuracy = 77.33%
Weaker 9 accuracy = 74.67%

AdaBoost accuracy = 100.00%, training completed!


使用鸢尾花测试数据集进行测试

In [12]:
results = ada.predict(XTEST)
scores = (results == YTEST)
print('Accuracy = {:3.2f}%'.format(Counter(scores)[True]/len(YTEST) * 100))

Accuracy = 100.00%


使用sklearn.ensemble.AdaBoostClassifier

- algorithm：该参数只对AdaBoostClassifier有效——scikit-learn实现了两种Adaboost分类算法，即SAMME和SAMME.R。两者的主要区别是弱学习器权重的度量，前者使用分类效果作为弱学习器的权重；后者使用分类预测概率作为弱学习器的权重，迭代一般比前者快。AdaBoostClassifier的默认算法是SAMME.R。需要注意的是，如果使用SAMME.R，则弱分类学习器的参数base_estimator必须限制为支持概率预测的分类器，而SAMME算法没有此限制；
- n_estimators： 该参数在AdaBoostClassifier和AdaBoostRegressor中都有效，它是弱学习器的最大迭代次数或弱学习器的最大数目。一般来说，n_estimators太小，容易欠拟合；n_estimators太大，又容易过拟合。一般选择一个适中的数值，默认是50。在实际调参的过程中，常常将n_estimators和下面介绍的参数learning_rate一起考虑；
- learning_rate: 该参数在AdaBoostClassifier和AdaBoostRegressor中都有效，表示每个弱学习器的权重衰减系数；
- base_estimator：该参数在AdaBoostClassifier和AdaBoostRegressor中都有效，表示弱分类学习器或者弱回归学习器。理论上，可以选择任何一个分类或者回归学习器，不过需要支持样本权重。常用学习器可以是CART决策树或者神经网络MLP；

In [13]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators = 10)
clf.fit(XTRAIN, YTRAIN)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=10, random_state=None)

In [14]:
clf.score(XTEST, YTEST)

1.0

回归提升树的实现

In [15]:
class BoostTree:
    def __init__(self, base):
        self.base = base #弱回归器
        
    def fit(self, X, Y, max_step):
        self.weaker = [] #存放弱回归器
        R = copy.deepcopy(Y) #残差
        for step in range(max_step): #训练弱回归器
            weaker = copy.deepcopy(self.base) #生成一个新弱回归器
            weaker.fit(X, R) #训练弱回归器
            results = weaker.predict(X)
            R = R - results #计算残差
            self.weaker.append(weaker)
            #计算误差
            f = np.sum(np.c_[[weaker.predict(X) for weaker in self.weaker]].T, axis = 1)
            print('Step {}, square loss {}'.format(step, np.linalg.norm(f - Y)**2))
        print('\nBoosting tree training completed!')
        
    def predict(self, X):
        return np.sum(np.c_[[weaker.predict(X) for weaker in self.weaker]].T, axis = 1)

验证例8.2

构建训练数据集

In [16]:
X = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Y = [5.56, 5.70, 5.91, 6.40, 6.80, 7.05, 8.90, 8.70, 9.00, 9.05]
XTEST = [1.2, 2.3, 3.4, 4.5, 5.6, 6.7, 7.8, 8.9, 9.5, 10.8]

In [17]:
from sklearn.tree import DecisionTreeRegressor
bt = BoostTree(DecisionTreeRegressor(max_depth = 1)) #使用回归决策树桩作为弱回归器
bt.fit(np.array(X).reshape(-1, 1), Y, 6)

Step 0, square loss 1.9300083333333338
Step 1, square loss 0.800675
Step 2, square loss 0.4780083333333336
Step 3, square loss 0.3055592592592599
Step 4, square loss 0.22891522633744946
Step 5, square loss 0.1721780649862837

Boosting tree training completed!


In [18]:
bt.predict(np.array(XTEST).reshape(-1, 1))

array([ 5.63      ,  5.63      ,  5.81831019,  6.55164352,  6.81969907,
        8.95016204,  8.95016204,  8.95016204,  8.95016204,  8.95016204])

使用sklearn.ensemble.AdaBoostRegressor

In [19]:
from sklearn.ensemble import AdaBoostRegressor
rsr = AdaBoostRegressor(loss = 'square', n_estimators = 6)
rsr.fit(np.array(X).reshape(-1, 1), Y)

AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='square',
         n_estimators=6, random_state=None)

In [20]:
print('Final square loss {}'.format(np.linalg.norm(rsr.predict(np.array(X).reshape(-1, 1)) - Y)**2))

Final square loss 0.04490000000000047


In [21]:
rsr.predict(np.array(XTEST).reshape(-1, 1))

array([ 5.63,  5.7 ,  5.91,  6.4 ,  7.05,  8.9 ,  8.9 ,  9.  ,  9.  ,  9.05])

使用sklearn.ensemble.GradientBoostingRegressor

In [22]:
from sklearn.ensemble import GradientBoostingRegressor
rsr = GradientBoostingRegressor(loss = 'ls', n_estimators = 6)
rsr.fit(np.array(X).reshape(-1, 1), Y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=6, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [23]:
print('Final square loss {}'.format(np.linalg.norm(rsr.predict(np.array(X).reshape(-1, 1)) - Y)**2))

Final square loss 5.424059630005812


In [24]:
rsr.predict(np.array(XTEST).reshape(-1, 1))

array([ 6.50762479,  6.54599735,  6.64125405,  6.89189235,  7.14105909,
        8.05341449,  7.95970269,  8.10027039,  8.10027039,  8.12369834])

参考文献

- https://github.com/wzyonggege/statistical-learning-method
- https://github.com/fengdu78/lihang-code/
- 李航。《统计学习方法》，清华大学出版社，2012年3月第1版；
- https://www.cnblogs.com/ScorpioLu/p/8295990.html
- https://zhuanlan.zhihu.com/p/59121403
- https://zhuanlan.zhihu.com/p/67186694